In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/sample_submission.csv
/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/all_data.csv
/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/test_public_expanded.csv
/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/test_private_expanded.csv
/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/toxicity_individual_annotations.csv
/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/train.csv
/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/identity_individual_annotations.csv
/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/test.csv


In [2]:
train =pd.read_csv('/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/train.csv')
test = pd.read_csv('/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/test.csv')
df_test = test['comment_text']
df = train[['target','comment_text']]

In [3]:
from imblearn.under_sampling import RandomUnderSampler
X = df['comment_text']
X = X.values.reshape(-1,1)
y = df['target']
y = np.where(y>=0.5,1.,0.)
undersample = RandomUnderSampler(sampling_strategy='majority')
X_sample, y_sample = undersample.fit_resample(X, y)

In [4]:
unique, counts = np.unique(y_sample, return_counts=True)
unique, counts

(array([0., 1.]), array([144334, 144334]))

In [5]:
!python -m spacy download en

⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB 47.5 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.2.0
    Uninstalling typing_extensions-4.2.0:
      Successfully uninstalled typing_extensions-4.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-io 0.21.0 requires tensorflow-io-gcs-filesystem==0.21.0, which is not installed.
tensorflow 2.6.4 requires absl-py~=0.10, but you have absl-py 1.0.0 which is incompatible.
tensorflow 2.6.4 requires numpy~=1.19.2, but you have numpy 1.21.6 which is incompatible.
tensorflow 2.6.4 requires six~=1.15.0, but you have six 1.16.0 which is incompatible.
tensorflow 2.6.4 requires wrapt~=1.12.1, but you have w

In [6]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [7]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X_sample.reshape(-1), y_sample, 
                                                    test_size=0.3, random_state=42)

In [8]:
from tqdm.auto import tqdm
x_train_v= np.zeros((len(x_train),96))
x_test_v= np.zeros((len(x_test),96))

for i, text in enumerate(tqdm( nlp.pipe(x_train) , total =len(x_train) )):
    x_train_v[i,:]=text.vector
    
for i, text in enumerate(tqdm ( nlp.pipe(x_test) , total= len(x_test))):
    x_test_v[i,:]=text.vector
    

  0%|          | 0/202067 [00:00<?, ?it/s]

  0%|          | 0/86601 [00:00<?, ?it/s]

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
svc = LinearSVC()
svc.fit(x_train_v,y_train)

In [ ]:
from sklearn import metrics

def print_report(clf, x_test, y_test):
    y_pred = clf.predict(x_test)
    report = metrics.classification_report(y_test, y_pred)
    print(report)
    print("accuracy: {:0.3f}".format(metrics.accuracy_score(y_test, y_pred)))

print_report(svc, x_test_v, y_test)

In [ ]:
reg = LogisticRegression()
reg.fit(x_train_v, y_train)

In [ ]:
print_report(reg, x_test_v, y_test)

In [25]:
from keras.models import Sequential
from keras.layers import Dense
model = Sequential()
model.add(Dense(units = 100, kernel_initializer = 'uniform', activation = 'relu', 
                input_dim = x_train_v.shape[1]))
model.add(Dense(units = 50, kernel_initializer = 'uniform', activation = 'relu'))
model.add(Dense(units = 25, kernel_initializer = 'uniform', activation = 'relu'))
model.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_21 (Dense)             (None, 100)               9700      
_________________________________________________________________
dense_22 (Dense)             (None, 50)                5050      
_________________________________________________________________
dense_23 (Dense)             (None, 25)                1275      
_________________________________________________________________
dense_24 (Dense)             (None, 1)                 26        
Total params: 16,051
Trainable params: 16,051
Non-trainable params: 0
_________________________________________________________________


In [26]:
model.fit(x_train_v, y_train,batch_size=32,epochs=40,validation_data=(x_test_v,y_test))

Epoch 1/40
6315/6315 [==============================] - 27s 4ms/step - loss: 0.6441 - accuracy: 0.6266 - val_loss: 0.6367 - val_accuracy: 0.6357
Epoch 2/40
6315/6315 [==============================] - 24s 4ms/step - loss: 0.6326 - accuracy: 0.6407 - val_loss: 0.6292 - val_accuracy: 0.6434
Epoch 3/40
6315/6315 [==============================] - 23s 4ms/step - loss: 0.6279 - accuracy: 0.6448 - val_loss: 0.6284 - val_accuracy: 0.6460
Epoch 4/40
6315/6315 [==============================] - 26s 4ms/step - loss: 0.6242 - accuracy: 0.6486 - val_loss: 0.6307 - val_accuracy: 0.6436
Epoch 5/40
6315/6315 [==============================] - 24s 4ms/step - loss: 0.6212 - accuracy: 0.6508 - val_loss: 0.6267 - val_accuracy: 0.6445
Epoch 6/40
6315/6315 [==============================] - 23s 4ms/step - loss: 0.6182 - accuracy: 0.6530 - val_loss: 0.6288 - val_accuracy: 0.6420
Epoch 7/40
6315/6315 [==============================] - 24s 4ms/step - loss: 0.6151 - accuracy: 0.6565 - val_loss: 0.6287 - val_ac

In [18]:
model.evaluate(x_test_v, y_test)

2707/2707 [==============================] - 9s 3ms/step - loss: 2.0883 - accuracy: 0.6030


[2.0882697105407715, 0.6029837727546692]

# try embeddings with ngram using cnn